In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import Code, Computer, load_node
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Int, Str, Bool
from aiida.work.workfunction import workfunction
from aiida.work.run import submit

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
import ase.build

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview
from collections import OrderedDict

from apps.surfaces.structure_browser import StructureBrowser
from apps.mcfitting.enework import EnergyWorkChain

# Energy Forces simulation

In [ ]:
def on_struct_change(c):
    global orig_structure
    orig_structure = None # disable event processing
    s = struct_browser.results.value
    if s:
        orig_structure = s
    update_view()
    
def update_view():
    with info_out:
        clear_output()

        # remove old components
        if hasattr(viewer, "component_0"):
            viewer.component_0.remove_ball_and_stick()
            viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        if orig_structure:
            atoms = orig_structure.get_ase()

            # add new component
            viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
            viewer.add_unitcell()
            viewer.center()

## Step 1: Select structures

In [ ]:
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
info_out = ipw.Output()

display(struct_browser)
display(ipw.VBox((viewer, info_out)))

## Step 2: Submit energy calculations

In [ ]:
# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    label = match[0]
    code = match[1]
    all_computers[label] = code

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

text_wavefunction = ipw.Text(placeholder='../asd.wfn',
                        description='Wavefunction',
                        style=style, layout=layout)

int_machines = ipw.IntText(value=40,
                           description='# Nodes',
                           style=style, layout=layout)

display(ipw.VBox([drop_computer, int_machines, text_wavefunction]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return

        cp2k_code = drop_computer.value
        struct = struct_browser.results.value
        wavefunction = Str(text_wavefunction.value)
        num_machines = Int(int_machines.value)
        gasphase = Bool(toggle_gasphase.value)

        outputs = submit(EnergyWorkChain,
                         cp2k_code=cp2k_code,
                         structure=struct,
                         wavefunction=wavefunction,
                         num_machines=num_machines,
                         gasphase=gasphase)
        print(outputs)



btn_submit = ipw.Button(description="Submit",
                        style=style, layout=layout)
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

In [ ]:
text_lower_n = ipw.IntText(description='Lower pk',
                           style=style, layout=layout)

text_upper_n = ipw.IntText(description='Upper pk',
                           style=style, layout=layout)

text_list = ipw.Text(description='All pks',
                     style=style, layout=layout)

toggle_gasphase = ipw.ToggleButton(description='Gas phase',
                                   style=style, layout=layout)

display(ipw.VBox([text_lower_n, text_upper_n, text_list, toggle_gasphase]))

In [ ]:
def on_submit_all(b):
    with submit_out_all:
        clear_output()
        if text_lower_n.value > 0:
            structures = range(text_lower_n.value, text_upper_n.value+1)
            print 'doing the range thingy'
        else:
            structures = [int(x) for x in text_list.value.split(' ')]
            print 'doing the list thingy'
        print "Calculating structures {} through {}.".format(min(structures), max(structures))
        if not drop_computer.value:
            print("Please select a computer.")
            return

        cp2k_code = drop_computer.value
        wavefunction = Str(text_wavefunction.value)
        num_machines = Int(int_machines.value)
        gasphase = Bool(toggle_gasphase.value)
        
        outputs = []
        
        for s in structures:
            struct = load_node(s)
            output = submit(EnergyWorkChain,
                            cp2k_code=cp2k_code,
                            structure=struct,
                            wavefunction=wavefunction,
                            num_machines=num_machines,
                            gasphase=gasphase)
            outputs.append(output)
            
        print('Starting {} calculations.'.format(len(outputs)))
        print(outputs)



btn_submit_all = ipw.Button(description="Submit range",
                            style=style, layout=layout)
btn_submit_all.on_click(on_submit_all)
submit_out_all = ipw.Output()
display(btn_submit_all, submit_out_all)

## Query the db

In [ ]:
qb = QueryBuilder()
qb.append(StructureData,
          filters = {
              'label': {'==': 'molecular circus'}
          },
          project = ['id', 'description']
)
res = qb.all()

In [ ]:
molecules = dict()
for r in res:
    try:
        molecules[r[1].split(' ')[0]]['n'] += 1
        molecules[r[1].split(' ')[0]]['pks'].append(r[0])
    except KeyError:
        molecules[r[1].split(' ')[0]] = {'n': 0, 'pks': [r[0],]}
        molecules[r[1].split(' ')[0]]['n'] = 1
        
for name, mol in molecules.items():
    molecules[name]['pk_min'] = min(molecules[name]['pks'])
    molecules[name]['pk_max'] = max(molecules[name]['pks'])
    print "{} goes from {} to {}".format(name, molecules[name]['pk_min'], molecules[name]['pk_max'])
    print "Total: {}".format(len(molecules[name]['pks']))
    print ' '.join([str(x) for x in molecules[name]['pks']])


**ft1-hexabenzocoronene**: /scratch/snx3000/editler/aiida_run_daint-s746/fb/d2/aaa5-6eac-40da-904a-444f91a2c58b/aiida-RESTART.wfn

**ft2-precursor**: /scratch/snx3000/editler/aiida_run_daint-s746/e1/b7/69a2-c143-442f-9192-144b9e3e8069/aiida-RESTART.wfn

**ft4-precursorI**: /scratch/snx3000/editler/aiida_run_daint-s746/d9/38/4c6d-e402-4034-b363-b659c771303c/aiida-RESTART.wfn

**ft3-precursorBr**: /scratch/snx3000/editler/aiida_run_daint-s746/4b/23/bebd-4f63-4a99-8119-860898dfa97e/aiida-RESTART.wfn

**ft5-PTCDA**: /scratch/snx3000/editler/aiida_run_daint-s746/75/9c/6fa6-4c67-4f20-a437-80f9a7c11d6d/aiida-RESTART.wfn

**ft6-terphenyl**: /scratch/snx3000/editler/aiida_run_daint-s746/38/69/c95a-47ad-4b5c-b161-217a72810420/aiida-RESTART.wfn

**ft7-pyrrole**: /scratch/snx3000/editler/aiida_run_daint-s746/aa/23/f3a6-bbdb-4101-b1fd-367922859d44/aiida-RESTART.wfn

# Change the descriptions

In [ ]:
from aiida.orm.calculation.work import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation

In [ ]:
qb = QueryBuilder()
qb.append(StructureData,
          filters = {
              'label': {'==': 'molecular circus'}
          },
          tag = 'mc',
          project = '*'
)
qb.append(WorkCalculation,
          descendant_of = 'mc',
          tag = 'mc_work',
          project = '*'
)
qb.append(Cp2kCalculation,
          output_of = 'mc_work',
          project = '*'
)
r = qb.all()

for res in r:
    res[2].description = res[0].description
    res[1].description = res[0].description